###  Practica final - Algoritmos de Optimización
- Autor: Raúl Capellán Fernández
- URL Github: https://github.com/RaulCapellanFernandez/AlgoritmosOptimizacion/blob/main/ActividadFinal/P.Final-Optimizaci%C3%B3n.ipynb

#### 2. Optimización escenas de doblaje:

Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las
tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben. No es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que se gasto por los
servicios de los actores de doblaje sea el menor posible.

In [1]:
import os.path
import pandas as pd
import numpy as np
import random 
import math
import copy 

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Lee el excel
path = "./datosDoblaje.xlsx"
norm_path = os.path.normpath(path) 
datosDoblaje = pd.read_excel(norm_path)

##### Primera solución -> Algoritmo Voraz
No será la mejor solución pero nos proporcionará una buena primera solución

En esta primera solución para este problema aplicamos un algoritmo recursivo que busca la mejor solución para cada paso en el que le introducimos el orden en el que se tienen que ejecutar los nodos, en este caso escenas.
Este algoritmo no nos va a dar una solución óptima pero nos da una primera solución con la que vamos a poder trabajar.

In [3]:
diaFinal = list()
posInterAnt = list()
posInterPost = list()

def optimizaOrdenDoblaje(diaOrden, actores, escena, matrizOcupacion):
    if(escena+1 == 31):
        print('YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS')
        #print('/////////////////////////////////////////////')
        #print('///////// Se ha tardado',diaOrden,'dias //////////////')
        #print('/////////////////////////////////////////////')
        diaFinal.append(diaOrden)
    else:
        total = 0
        for i in range(actores):
            actorPos = 'ACT' + str(i+1)
            participacionesActor = matrizOcupacion[i][diaOrden]
            
            if(datosDoblaje.iloc[escena][actorPos]== 1):
                if(participacionesActor < 6):
                    matrizOcupacion[i][diaOrden] = matrizOcupacion[i][diaOrden] + 1
                    total = total + 1
                    #print('El dia',diaOrden,'El actor',actorPos,'puede trabajar en la escena',escena+1)
                    #print('Este actor ha trabajado hoy en',matrizOcupacion[i][diaOrden],'escenas')
                #else:
                    #print('El dia',diaOrden,'El actor',actorPos,'NO puede trabajar en la escena',escena+1)
                    #print('Este actor ha trabajado hoy en',matrizOcupacion[i][diaOrden],'escenas')
        if(total == datosDoblaje.iloc[escena]['Total']):
            #print('////////////////////////////////////////////////////////////')
            #print('La escena',escena+1,'se va a llevar a cabo el dia',diaOrden)
            #print('////////////////////////////////////////////////////////////')
            optimizaOrdenDoblaje(diaOrden, actores, escena+1, matrizOcupacion)
        else:
            #print('**************************************************************')
            #print('La escena',escena+1,'NO se va a llevar a cabo el dia',diaOrden)
            #print('**************************************************************')
            optimizaOrdenDoblaje(diaOrden+1, actores, escena, matrizOcupacion)

In [4]:
actores  = 10
escena  = 0
diaOrden =1

matrizOcupacion = np.zeros((10, 40))

optimizaOrdenDoblaje(diaOrden, actores, escena, matrizOcupacion)

YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS


#### Segunda solución -> Algoritmos Heurísticos -> Recocido simulado

Ya que no encuentro ningún algoritmo óptimo para este problema nos encontramos con la solución de utilizar algoritmos heuristicos, los cuales nos permiten obetener una buena solución en un tiempo aceptable.
En este caso utilizamos el algoritmo de recocido simulado el cual usamos en la práctica guiada 3, la cual la adaptamos para nuestro problema.
A parte le incluimos la opción de volver a la iteración anterior si en la actual no mejoramos, lo cual nos permite obviar las soluciones en las que nuestro algoritmo empeora.
Finalmente no llegamos a obtener una gran mejora con este algoritmo ya que tiene que haber una gran mejora para que mejore este algoritmo ya que tiene que reducir el número de días.
Tras un par de decenas de ejecuciones, llegamos a la conclusión que la primera solución obtenida es un mínimo local bastante bueno.

In [5]:
def genera_vecina_aleatorio(solucion_referencia):
    #Coge dos nodos aleatorios y los intercambia
    i,j = sorted(random.sample( range(0,len(solucion_referencia)) , 2))
    #Modifica el dataframe cambiando esos dos nodos de posicion
    cambia_orden_nodos(i,j)
    
    posInterAnt.append(i)
    posInterPost.append(j)
    
    aux = solucion_referencia[i]
    solucion_referencia[i] = solucion_referencia[j]
    solucion_referencia[j] = aux
    return solucion_referencia

In [6]:
def rollBack(i, j, solucion_referencia):
    cambia_orden_nodos(i,j)
    
    aux = solucion_referencia[i]
    solucion_referencia[i] = solucion_referencia[j]
    solucion_referencia[j] = aux
    
    return solucion_referencia

In [7]:
def cambia_orden_nodos(nodo1, nodo2):
    aux = datosDoblaje.iloc[nodo1]
    datosDoblaje.iloc[nodo1] = datosDoblaje.iloc[nodo2]
    datosDoblaje.iloc[nodo2] = aux

In [8]:
def probabilidad(T,d):
    if random.random() <  math.exp( -1*d / T)  :
        return True
    else:
        return False

In [9]:
#Funcion de descenso de temperatura
def bajar_temperatura(T):
    return T*0.99

In [10]:
def recorrido_simulado(TEMPERATURA):
    solucion_referencia = list()
    #Se utiliza  como solucion de referencia la calculada previamente
    for i in range(30): 
        solucion_referencia.append(i)
    matrizOcupacion = np.zeros((10, 40))
    optimizaOrdenDoblaje(1, 10, 0, matrizOcupacion)
    distancia_referencia = diaFinal[-1]
      
    mejor_solucion = list()
    mejor_distancia = 1000
    
    N=0
    while TEMPERATURA > .0001:
        N+=1
        #Genera una solución vecina
        vecina = genera_vecina_aleatorio(solucion_referencia)
        
        #Calcula el numero de dias
        matrizOcupacion = np.zeros((10, 40))
        optimizaOrdenDoblaje(1, 10, 0, matrizOcupacion)
        distancia_vecina = diaFinal[-1]
        
        #Si es la mejor solución de todas se guarda(siempre!!!)
        if distancia_vecina < mejor_distancia:
            print('Se ha tardado', distancia_vecina,'dias')
            print('La mejor solucion es', vecina)
            mejor_solucion = vecina
            mejor_distancia = distancia_vecina
            
        if distancia_vecina < distancia_referencia or probabilidad(TEMPERATURA, abs(distancia_referencia - distancia_vecina) ) :
            print('Se ha tardado', distancia_vecina,'dias')
            print('No mejor solucion es', vecina)
            
            if(distancia_referencia < distancia_vecina):
                solucion_referencia = rollBack(posInterAnt[-1], posInterPost[-1], solucion_referencia)
                vecina = genera_vecina_aleatorio(solucion_referencia)
                
                matrizOcupacion = np.zeros((10, 40))
                optimizaOrdenDoblaje(1, 10, 0, matrizOcupacion)
                distancia_vecina = diaFinal[-1]
                
                print('Hemos vuelto a la solucion anterior')
                print('Se ha tardado', distancia_vecina,'dias')
                print('ROLLBACK solucion', vecina)
            
            solucion_referencia = copy.deepcopy(vecina)
            distancia_referencia = distancia_vecina
        
        TEMPERATURA = bajar_temperatura(TEMPERATURA)
    
    print("La mejor solución encontrada es " , end="")
    print(mejor_solucion)
    print("con una distancia total de " , end="")
    print(mejor_distancia)
    return mejor_solucion

In [11]:
solucion_referencia = list()
    #Se utiliza  como solucion de referencia la calculada previamente
for i in range(30): 
    solucion_referencia.append(i)
    
sol  = recorrido_simulado(10000000)

YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
La mejor solucion es [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 28, 20, 21, 22, 23, 24, 25, 26, 27, 19, 29]
Se ha tardado 4 dias
No mejor solucion es [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 28, 20, 21, 22, 23, 24, 25, 26, 27, 19, 29]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 18, 15, 16, 17, 14, 28, 20, 21, 22, 23, 24, 25, 26, 27, 19, 29]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [0, 1, 23, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 18, 15, 16, 17, 14, 28, 20, 21, 22, 2, 24, 25, 26, 27, 19, 29]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [0, 1, 23, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 18, 26, 16, 17, 14, 28, 20, 21, 22, 2, 24, 25, 15, 27, 19, 29]
YA HEMOS ACABADO DE ORGAN

YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [21, 1, 25, 24, 4, 16, 11, 6, 12, 9, 15, 10, 20, 0, 2, 7, 5, 8, 26, 23, 19, 27, 29, 3, 28, 13, 18, 14, 22, 17]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [21, 1, 25, 24, 4, 16, 11, 6, 12, 9, 15, 10, 20, 0, 2, 7, 5, 8, 26, 23, 17, 27, 29, 3, 28, 13, 18, 14, 22, 19]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [21, 1, 25, 24, 4, 16, 11, 6, 12, 9, 15, 28, 20, 0, 2, 7, 5, 8, 26, 23, 17, 27, 29, 3, 10, 13, 18, 14, 22, 19]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [12, 1, 25, 24, 4, 16, 11, 6, 21, 9, 15, 28, 20, 0, 2, 7, 5, 8, 26, 23, 17, 27, 29, 3, 10, 13, 18, 14, 22, 19]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [12, 1, 25, 24, 4, 16, 11, 6, 21, 9, 2, 28, 20, 0, 15, 7, 5, 8, 26, 23, 17, 27, 29, 3, 10, 13, 18, 14, 22, 19]
YA HEMOS ACABADO DE ORGAN

YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [6, 17, 15, 28, 3, 23, 24, 2, 21, 16, 11, 25, 20, 5, 27, 0, 1, 12, 29, 7, 9, 13, 14, 18, 10, 4, 8, 22, 26, 19]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [6, 17, 15, 28, 3, 23, 24, 2, 21, 16, 11, 25, 20, 5, 27, 0, 1, 12, 29, 7, 9, 13, 14, 18, 10, 4, 22, 8, 26, 19]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [6, 17, 5, 28, 3, 23, 24, 2, 21, 16, 11, 25, 20, 15, 27, 0, 1, 12, 29, 7, 9, 13, 14, 18, 10, 4, 22, 8, 26, 19]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [6, 17, 5, 28, 3, 27, 24, 2, 21, 16, 11, 25, 20, 15, 23, 0, 1, 12, 29, 7, 9, 13, 14, 18, 10, 4, 22, 8, 26, 19]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [6, 17, 5, 28, 3, 27, 24, 2, 21, 16, 11, 13, 20, 15, 23, 0, 1, 12, 29, 7, 9, 25, 14, 18, 10, 4, 22, 8, 26, 19]
YA HEMOS ACABADO DE ORGAN

YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [13, 8, 5, 15, 0, 14, 4, 2, 26, 12, 22, 23, 19, 29, 18, 7, 17, 11, 16, 28, 9, 25, 3, 1, 27, 6, 24, 10, 21, 20]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [13, 8, 5, 15, 0, 14, 4, 2, 26, 12, 22, 23, 19, 29, 18, 7, 17, 11, 16, 28, 9, 25, 3, 20, 27, 6, 24, 10, 21, 1]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [13, 8, 5, 15, 2, 14, 4, 0, 26, 12, 22, 23, 19, 29, 18, 7, 17, 11, 16, 28, 9, 25, 3, 20, 27, 6, 24, 10, 21, 1]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [13, 8, 5, 15, 2, 14, 4, 0, 23, 12, 22, 26, 19, 29, 18, 7, 17, 11, 16, 28, 9, 25, 3, 20, 27, 6, 24, 10, 21, 1]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [13, 14, 5, 15, 2, 8, 4, 0, 23, 12, 22, 26, 19, 29, 18, 7, 17, 11, 16, 28, 9, 25, 3, 20, 27, 6, 24, 10, 21, 1]
YA HEMOS ACABADO DE ORGAN

YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [17, 7, 24, 26, 13, 28, 9, 2, 15, 14, 23, 25, 0, 6, 29, 19, 11, 8, 3, 27, 16, 1, 21, 22, 4, 5, 12, 10, 18, 20]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [17, 7, 24, 26, 13, 28, 9, 2, 15, 14, 23, 25, 0, 6, 29, 19, 11, 8, 5, 27, 16, 1, 21, 22, 4, 3, 12, 10, 18, 20]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [17, 7, 24, 26, 13, 28, 9, 2, 27, 14, 23, 25, 0, 6, 29, 19, 11, 8, 5, 15, 16, 1, 21, 22, 4, 3, 12, 10, 18, 20]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [17, 7, 24, 26, 13, 28, 9, 2, 27, 14, 23, 25, 0, 6, 29, 19, 11, 8, 5, 15, 12, 1, 21, 22, 4, 3, 16, 10, 18, 20]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [17, 7, 24, 26, 13, 28, 9, 2, 27, 14, 23, 25, 0, 21, 29, 19, 11, 8, 5, 15, 12, 1, 6, 22, 4, 3, 16, 10, 18, 20]
YA HEMOS ACABADO DE ORGAN

...

YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [18, 7, 26, 9, 17, 13, 24, 19, 15, 23, 2, 14, 29, 6, 4, 1, 0, 12, 8, 22, 20, 25, 3, 11, 27, 16, 21, 5, 10, 28]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [18, 7, 26, 9, 17, 13, 24, 19, 15, 23, 2, 14, 29, 6, 4, 1, 0, 12, 8, 22, 20, 11, 3, 25, 27, 16, 21, 5, 10, 28]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [18, 7, 4, 9, 17, 13, 24, 19, 15, 23, 2, 14, 29, 6, 26, 1, 0, 12, 8, 22, 20, 11, 3, 25, 27, 16, 21, 5, 10, 28]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [18, 7, 4, 9, 17, 13, 24, 19, 15, 23, 20, 14, 29, 6, 26, 1, 0, 12, 8, 22, 2, 11, 3, 25, 27, 16, 21, 5, 10, 28]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [18, 7, 4, 9, 17, 13, 24, 19, 15, 23, 20, 14, 29, 6, 26, 1, 0, 12, 8, 28, 2, 11, 3, 25, 27, 16, 21, 5, 10, 22]
YA HEMOS ACABADO DE ORGAN

YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [15, 7, 1, 9, 29, 13, 18, 23, 8, 11, 16, 24, 2, 12, 27, 0, 21, 25, 14, 26, 3, 17, 19, 22, 28, 5, 6, 20, 10, 4]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [15, 7, 1, 9, 29, 13, 18, 23, 8, 11, 16, 24, 2, 12, 27, 0, 21, 25, 14, 26, 19, 17, 3, 22, 28, 5, 6, 20, 10, 4]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [15, 7, 1, 9, 29, 13, 18, 23, 8, 11, 16, 24, 2, 12, 27, 0, 21, 3, 14, 26, 19, 17, 25, 22, 28, 5, 6, 20, 10, 4]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [15, 7, 1, 9, 2, 13, 18, 23, 8, 11, 16, 24, 29, 12, 27, 0, 21, 3, 14, 26, 19, 17, 25, 22, 28, 5, 6, 20, 10, 4]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [15, 7, 1, 9, 2, 13, 18, 23, 8, 11, 16, 24, 29, 12, 27, 14, 21, 3, 0, 26, 19, 17, 25, 22, 28, 5, 6, 20, 10, 4]
YA HEMOS ACABADO DE ORGAN

YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [15, 12, 4, 25, 23, 27, 18, 14, 26, 19, 3, 22, 21, 8, 17, 10, 13, 28, 7, 1, 16, 9, 2, 0, 6, 5, 29, 24, 20, 11]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [15, 12, 4, 25, 23, 27, 18, 14, 29, 19, 3, 22, 21, 8, 17, 10, 13, 28, 7, 1, 16, 9, 2, 0, 6, 5, 26, 24, 20, 11]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [15, 12, 4, 17, 23, 27, 18, 14, 29, 19, 3, 22, 21, 8, 25, 10, 13, 28, 7, 1, 16, 9, 2, 0, 6, 5, 26, 24, 20, 11]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [15, 12, 4, 20, 23, 27, 18, 14, 29, 19, 3, 22, 21, 8, 25, 10, 13, 28, 7, 1, 16, 9, 2, 0, 6, 5, 26, 24, 17, 11]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [10, 12, 4, 20, 23, 27, 1, 14, 29, 19, 3, 22, 21, 8,

YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [8, 28, 26, 13, 5, 27, 10, 1, 7, 20, 29, 18, 4, 2, 25, 14, 6, 3, 11, 0, 12, 23, 16, 15, 21, 17, 24, 22, 19, 9]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [8, 28, 26, 13, 5, 27, 10, 1, 14, 20, 29, 18, 4, 2, 25, 7, 6, 3, 11, 0, 12, 23, 16, 15, 21, 17, 24, 22, 19, 9]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [8, 28, 26, 13, 5, 27, 10, 1, 14, 20, 29, 18, 4, 2, 25, 0, 6, 3, 11, 7, 12, 23, 16, 15, 21, 17, 24, 22, 19, 9]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [8, 28, 26, 13, 5, 27, 10, 1, 14, 20, 15, 18, 4, 2, 25, 0, 6, 3, 11, 7, 12, 23, 16, 29, 21, 17, 24, 22, 19, 9]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [8, 28, 26, 13, 5, 27, 10, 1, 14, 22, 15, 18, 4, 2, 25, 0, 6, 3, 11, 7, 12, 23, 16, 29, 21, 17, 24, 20, 19, 9]
YA HEMOS ACABADO DE ORGAN

YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [18, 5, 1, 4, 29, 13, 3, 14, 22, 19, 15, 25, 21, 2, 8, 17, 26, 10, 16, 11, 24, 23, 7, 12, 27, 0, 20, 28, 6, 9]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [18, 5, 1, 4, 29, 13, 3, 14, 22, 19, 15, 10, 21, 2, 8, 17, 26, 25, 16, 11, 24, 23, 7, 12, 27, 0, 20, 28, 6, 9]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [18, 5, 1, 4, 29, 13, 3, 14, 22, 19, 15, 10, 21, 2, 26, 17, 8, 25, 16, 11, 24, 23, 7, 12, 27, 0, 20, 28, 6, 9]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [18, 5, 1, 4, 29, 13, 3, 14, 22, 19, 15, 10, 21, 2, 26, 17, 8, 9, 16, 11, 24, 23, 7, 12, 27, 0, 20, 28, 6, 25]
YA HEMOS ACABADO DE ORGANIZAR LAS ESCENAS
Se ha tardado 4 dias
No mejor solucion es [18, 5, 24, 4, 29, 13, 3, 14, 22, 19, 15, 10, 21, 2, 26, 17, 8, 9, 16, 11, 1, 23, 7, 12, 27, 0, 20, 28, 6, 25]
YA HEMOS ACABADO DE ORGAN